In [1]:
!pip install selenium


In [12]:
!pip install pymysql


Data Scarping Code


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re
import pymysql as mysql

options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

genre_list = ['horror', 'drama', 'thriller', 'romance','documentary']

for genre in genre_list:
    print(f"Processing genre: {genre}")
    url = f"https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres={genre}"
    driver.get(url)
    time.sleep(3)

    
    while True:
        try:
            load_more = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'ipc-see-more__text') and contains(text(), 'more')]"))
            )
            driver.execute_script("arguments[0].click();", load_more)
            time.sleep(2)
        except:
            break

    
    movie_containers = driver.find_elements(By.XPATH, "//div[contains(@class, 'ipc-metadata-list-summary-item')]")
    
    data = []
    for movie in movie_containers:
        try:
         
            title = movie.find_element(By.CSS_SELECTOR, "h3.ipc-title__text").text.strip()
            title = re.sub(r'^\d+\.\s*', '', title)  

            
            meta_spans = movie.find_elements(By.CSS_SELECTOR, "span.dli-title-metadata-item")
            year = meta_spans[0].text if len(meta_spans) > 0 else ""
            if len(meta_spans) > 1:
                duration_raw = meta_spans[1].text
                hours = 0
                minutes = 0
                h_match = re.search(r'(\d+)h', duration_raw)
                if h_match:
                    hours = int(h_match.group(1))
                m_match = re.search(r'(\d+)m', duration_raw)
                if m_match:
                    minutes = int(m_match.group(1))
                duration = hours * 60 + minutes
            else:
                duration = ""

            try:
                rating = movie.find_element(By.CSS_SELECTOR, "span.ipc-rating-star--rating").text
            except:
                rating = ""

            
            try:
                vote_text = movie.find_element(By.CSS_SELECTOR, "span.ipc-rating-star--voteCount").text
                vote_count = re.sub(r'[^\dKk]', '', vote_text).upper()
                
                if 'K' in vote_count:
                    vote_count = int(float(vote_count.replace('K', '')) * 1000)
                elif 'M' in vote_count:
                    vote_count = int(float(vote_count.replace('M', '')) * 1_000_000)
                else:
                    vote_count = int(vote_count)
            except:
                vote_count = ""

            data.append({
                "Movie Name": title,
                "Year": year,
                "Duration": duration,
                
                "Rating": rating,
                "Vote Count": vote_count,
                "Genre": genre
            })

        except Exception as e:
            print(e)
            continue

    
    df = pd.DataFrame(data)
    df.drop_duplicates(inplace=True)
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce').fillna(0).astype(int)
    df['Duration'] = pd.to_numeric(df['Duration'], errors='coerce').fillna(0).astype(int)
    df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce').astype(float)
    df['Vote Count'] = pd.to_numeric(df['Vote Count'], errors='coerce').fillna(0).astype(int)
    df.to_csv(f"{genre}_2024_movies.csv", index=False)
    print(f"Saved {len(df)} records to {genre}_2024_movies.csv")

driver.quit()

Processing genre: horror
Saved 1743 records to horror_2024_movies.csv
Processing genre: drama
Saved 6387 records to drama_2024_movies.csv
Processing genre: thriller
Saved 2227 records to thriller_2024_movies.csv
Processing genre: romance
Saved 1206 records to romance_2024_movies.csv
Processing genre: documentary
Saved 5560 records to documentary_2024_movies.csv


Combining and Cleaning Code

In [ ]:

csv_files = [
    r'C:\Users\judej\OneDrive\Desktop\Scraping\documentary_2024_movies.csv',
    r'C:\Users\judej\OneDrive\Desktop\Scraping\drama_2024_movies.csv',
    r'C:\Users\judej\OneDrive\Desktop\Scraping\horror_2024_movies.csv',
    r'C:\Users\judej\OneDrive\Desktop\Scraping\romance_2024_movies.csv',
    r'C:\Users\judej\OneDrive\Desktop\Scraping\thriller_2024_movies.csv'
]

dataframes = [pd.read_csv(file) for file in csv_files]
combined = pd.concat(dataframes, ignore_index=True)


combined.dropna(subset=['Movie Name'], inplace=True)
combined = combined[combined['Movie Name'].str.strip() != '']


duplicate_titles = combined['Movie Name'].value_counts()
duplicate_titles = duplicate_titles[duplicate_titles > 1].index
combined = combined[~combined['Movie Name'].isin(duplicate_titles)].copy()


combined['Year'] = pd.to_numeric(combined['Year'], errors='coerce')
combined.loc[combined['Year'] == 0, 'Year'] = np.nan
combined['Year'] = combined['Year'].fillna(2024).astype(int)


combined['Duration'] = pd.to_numeric(combined['Duration'], errors='coerce')
combined.loc[combined['Duration'] == 0, 'Duration'] = np.nan


duration_means = combined.groupby('Genre')['Duration'].transform('mean')
combined['Duration'] = combined['Duration'].fillna(duration_means)
combined['Duration'] = combined['Duration'].fillna(combined['Duration'].mean())
combined['Duration'] = combined['Duration'].round().astype(int)


combined['Rating'] = pd.to_numeric(combined['Rating'], errors='coerce')
combined['Vote Count'] = pd.to_numeric(combined['Vote Count'], errors='coerce')


combined.loc[combined['Rating'] == 0, 'Rating'] = np.nan
combined.loc[combined['Vote Count'] == 0, 'Vote Count'] = np.nan


rating_means = combined.groupby('Genre')['Rating'].transform('mean')
combined['Rating'] = combined['Rating'].fillna(rating_means)
combined['Rating'] = combined['Rating'].fillna(combined['Rating'].mean())


vote_means = combined.groupby('Genre')['Vote Count'].transform('mean')
combined['Vote Count'] = combined['Vote Count'].fillna(vote_means)
combined['Vote Count'] = combined['Vote Count'].fillna(combined['Vote Count'].mean())


combined.dropna(subset=['Rating', 'Vote Count'], how='all', inplace=True)


combined['Vote Count'] = combined['Vote Count'].round().astype(int)
combined['Rating'] = combined['Rating'].round(1).astype(float)


combined = combined[(combined['Rating'] <= 10) & (combined['Rating'] >= 1)]
combined = combined[combined['Duration'] > 10]  # e.g., remove durations too low


combined.reset_index(drop=True, inplace=True)
combined.to_csv('combined_cleaned.csv', index=False)

print(f"Final record count: {len(combined)}")


df = pd.read_csv('combined_cleaned.csv')


Final record count: 13007


Debuging 

In [ ]:
df1 = pd.read_csv('combined_cleaned.csv')
print(df.dtypes)
dfc = df.count()
print(dfc)
print("Duration == 0:", (df['Duration'] == 0).sum())
print("Rating == 0:", (df['Rating'] == 0).sum())
print("Vote count == 0:", (df['Vote Count'] == 0).sum())
print("Duration == 0:", (df['Year'] == 0).sum())


Movie Name     object
Year            int64
Duration        int64
Rating        float64
Vote Count      int64
Genre          object
dtype: object
Movie Name    13007
Year          13007
Duration      13007
Rating        13007
Vote Count    13007
Genre         13007
dtype: int64
Number of rows where Duration == 0: 0
Number of rows where Duration == 0: 0
Number of rows where Duration == 0: 0
Number of rows where Duration == 0: 0


MY SQL Insertion

In [48]:
df = pd.read_csv(r'C:\Users\judej\OneDrive\Desktop\Scraping\combined_cleaned.csv')


connection = mysql.connect(
    host='localhost',
    user='root',
    password='root',
    database='scrape_db',  
    port=3306
)

cursor = connection.cursor()


insert_query = """
INSERT INTO movies (movie_name, year, duration, rating, vote_count, genre)
VALUES (%s, %s, %s, %s, %s, %s);
"""


for _, row in df.iterrows():
    cursor.execute(insert_query, (
        row['Movie Name'],
        int(row['Year']),
        int(row['Duration']),
        float(row['Rating']),
        int(row['Vote Count']),
        row['Genre']
    ))


connection.commit()
cursor.close()
connection.close()

print("Data insertion done.")

Data insertion done.
